In [125]:
import pandas as pd
columns = ["text", "language", "thread_title", "spam_score", "replies_count", "participants_count", "likes", "comments", "shares", "type"]
df = pd.read_csv("fake_news_dataset.csv", usecols=columns)

In [126]:
df.head()

,text,language,thread_title,spam_score,replies_count,participants_count,likes,comments,shares,type
0,Print They should pay all the back all the mon...,english,Muslims BUSTED: They Stole Millions In Gov’t B...,0.000,0,1,0,0,0,bias
1,Why Did Attorney General Loretta Lynch Plead T...,english,Re: Why Did Attorney General Loretta Lynch Ple...,0.000,0,1,0,0,0,bias
2,Red State : \nFox News Sunday reported this mo...,english,BREAKING: Weiner Cooperating With FBI On Hilla...,0.000,0,1,0,0,0,bias
3,Email Kayla Mueller was a prisoner and torture...,english,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,0.068,0,0,0,0,0,bias
4,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,english,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,0.865,0,0,0,0,0,bias


In [127]:
df = df[df["language"]=='english']
df = df.dropna()

In [128]:
df = df.drop("language", axis=1)

In [129]:
features = 0
feature_map = {}
def addFeature(name):
    if name not in feature_map:
        global features
        feature_map[name] = features
        features+=1
        
addFeature("fake")
addFeature("real")

In [130]:
def articleType(row):
    if row["type"]=='fake':
        return feature_map["fake"]
    else:
        return feature_map["real"]

In [131]:
df["type"] = df.apply(articleType, axis=1)

In [132]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df)

In [133]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizerText = TfidfVectorizer()
vectorizerTitle = TfidfVectorizer()

In [134]:
vectorizedText = vectorizerText.fit_transform(df_train.pop("text").values)

In [135]:
vectorizedTitle = vectorizerTitle.fit_transform(df_train.pop("thread_title").values)

In [170]:
from scipy import sparse
spam_score_train = sparse.csr_matrix(df_train["spam_score"].values).transpose()
replies_count_train = sparse.csr_matrix(df_train["replies_count"].values).transpose()
participants_count_train = sparse.csr_matrix(df_train["participants_count"].values).transpose()
likes_train = sparse.csr_matrix(df_train["likes"].values).transpose()
comments_train = sparse.csr_matrix(df_train["comments"].values).transpose()
shares_train = sparse.csr_matrix(df_train["shares"].values).transpose()

In [177]:
from scipy.sparse import hstack
X_train = hstack([vectorizedText, vectorizedTitle,spam_score_train,replies_count_train,
              participants_count_train,likes_train,comments_train,shares_train])
y_train = df_train.pop("type").values

In [178]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()

In [179]:
clf.fit(X_train, y_train)

C:\Users\ETsukerman\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [180]:
vectorizedText_test = vectorizerText.transform(df_test.pop("text").values)
vectorizedTitle_test = vectorizerTitle.transform(df_test.pop("thread_title").values)

In [181]:
spam_score_test = sparse.csr_matrix(df_test["spam_score"].values).transpose()
replies_count_test = sparse.csr_matrix(df_test["replies_count"].values).transpose()
participants_count_test = sparse.csr_matrix(df_test["participants_count"].values).transpose()
likes_test = sparse.csr_matrix(df_test["likes"].values).transpose()
comments_test = sparse.csr_matrix(df_test["comments"].values).transpose()
shares_test = sparse.csr_matrix(df_test["shares"].values).transpose()

In [182]:
X_test = hstack([vectorizedText_test, vectorizedTitle_test,spam_score_test,replies_count_test,
              participants_count_test,likes_test,comments_test,shares_test])
y_test = df_test.pop("type").values

In [184]:
clf.score(X_test, y_test)

0.9977324263038548